#

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

In [2]:
# tmcs = pd.read_excel('data/0.xlsx', sheet_name='TMCs', usecols=['compounds','CurrentDensity','overpotential']).loc[:49, :]
new_tmcs = pd.read_csv('new_TMCs_0815.csv')['compound']
cols_tm = ['element','AN','AW','CR','AR','VR','AV','FIE','SIE','EA','EN','Nd','Nv','D','EC','TC']
tm = pd.read_excel('data/0.xlsx', sheet_name='TM_descriptors', usecols=cols_tm).fillna(0,inplace = False).replace('/', 0, inplace=False)
cols_c = ['element','AN','AW','CR','AR','VR','AV','FIE','SIE','EA','EN','Np','Nv','D']
c =  pd.read_excel('data/0.xlsx', sheet_name='C_descriptors', usecols=cols_c).fillna(0,inplace = False)

In [3]:
# 拆分化合物
import re
import json

def parse_compound(compound):
    matches = re.findall(r'([A-Z][a-z]*)(\d*\.*\d*)', compound)
    result = {}
    for match in matches:
        element, count = match
        count = float(count) if count else 1.0  # 如果没有数字，默认为1.0
        result[element] = count
    
    return result

def get_res(compound_name):
    parsed_result = parse_compound(compound_name)
    return parsed_result

In [4]:
train_data = []
for index, compound in enumerate(new_tmcs.values):
    compound_feat = [compound]
    compound = get_res(compound)
    keys = list(compound.keys())
    val = list(compound.values())
    if len(keys) > 2:
        A_key, A_index = keys[0], val[0]
        B_key, B_index = keys[1], val[1]
        A_feat = tm[tm['element']==A_key].values.flatten()[1:] * float(A_index)
        B_feat = tm[tm['element']==B_key].values.flatten()[1:] * float(B_index)
        C_feat = c[c['element']==keys[2]].values.flatten()[1:] * float(val[2])
        AB_feat = np.array([a + b for a, b in zip(A_feat, B_feat)])
        compound_feat += list(AB_feat) + list(C_feat) + list([500])
    else:
        AB_key = keys[0]
        AB_index = val[0]
        AB_feat = tm[tm['element']==AB_key].values.flatten()[1:] * float(AB_index)
        C_feat = c[c['element']==keys[1]].values.flatten()[1:] * float(val[1])
        compound_feat += list(AB_feat) + list(C_feat) + list([500])
    train_data.append(compound_feat)

In [6]:
predict_data = np.array(train_data)[:, 1:]
predict_data.shape

(15012, 29)

In [7]:
print(predict_data.astype(float))

[[2.1000000e+01 4.4955900e+01 1.7000000e+00 ... 6.0000000e+00
  1.4290000e-03 5.0000000e+02]
 [2.1000000e+01 4.4955900e+01 1.7000000e+00 ... 1.2000000e+01
  2.8580000e-03 5.0000000e+02]
 [2.1000000e+01 4.4955900e+01 1.7000000e+00 ... 1.8000000e+01
  4.2870000e-03 5.0000000e+02]
 ...
 [5.2500000e+01 1.2640675e+02 1.3400000e+00 ... 1.2000000e+01
  3.9200000e+00 5.0000000e+02]
 [5.3000000e+01 1.2794985e+02 1.3100000e+00 ... 1.2000000e+01
  3.9200000e+00 5.0000000e+02]
 [5.3500000e+01 1.2783325e+02 1.3000000e+00 ... 1.2000000e+01
  3.9200000e+00 5.0000000e+02]]


In [8]:
# regressor  =RandomForestRegressor(n_estimators=100, random_state=21)
from xgboost import XGBRegressor
model=XGBRegressor()
with open('model/XGBoost_tuning_5.pkl', 'rb') as f:
    model = pickle.load(f) 
y_pred = model.predict(predict_data.astype(float))

In [7]:
# regressor  =RandomForestRegressor(n_estimators=100, random_state=21)
with open('model/rf_tuning_4.pkl', 'rb') as f:
    model = pickle.load(f)

y_pred = model.predict(predict_data)

In [9]:
index = np.array(train_data)[:,:1]
y_pred = np.array(y_pred).reshape(-1, 1)
result = np.concatenate((index, y_pred), axis=1)
sorted_indices = np.argsort(result[:, 1])
sorted_compound = result[sorted_indices]
col = ['compound', 'overpotential']
df_sorted_compound = pd.DataFrame(sorted_compound, columns=col)
import os
if not os.path.exists('predict/'):
    os.makedirs('predict/')
df_sorted_compound.to_csv('predict/predict_500_XGB_0815.csv', index=False)